### cohere modeli hfden upload etmeyi deniyoruz

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from transformers import AutoTokenizer, AutoModel
import os
from dotenv import load_dotenv

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))
hf_token = os.getenv("HF_API_KEY")  

tokenizer = AutoTokenizer.from_pretrained(
    "Cohere/Cohere-embed-multilingual-v3.0", 
    token=hf_token
)
model = AutoModel.from_pretrained(
    "Cohere/Cohere-embed-multilingual-v3.0",
    token=hf_token,
    torch_dtype="auto"
)
# Örnek metinler
texts = ["Bu bir test cümlesidir.", "Mahkeme kararını verdi."]

# Tokenize et
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Modeli kullanarak embedding al
with torch.no_grad():
    outputs = model(**inputs)
    # Burada last_hidden_state'den mean pooling yapıyoruz
    embeddings = outputs.last_hidden_state.mean(dim=1)

print("Embedding boyutu:", embeddings.shape)
print("İlk embedding örneği:", embeddings[0])


In [ ]:
!echo $HF_API_KEY

### gemma hf

In [1]:
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv
print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))
# Hugging Face API key ortam değişkeninden çek
hf_token = os.getenv("HF_API_KEY")

# Modeli token ile yükle
model = SentenceTransformer(
    "google/embeddinggemma-300m",
    token=hf_token  # API key burada
)

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]

embeddings = model.encode(sentences)

# Benzerlik hesapla
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)


/home/yapayzeka/ahsen_bulbul/ahsen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
torch.Size([4, 4])


In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os
from dotenv import load_dotenv

# .env dosyasını yükle
load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env")

# Hugging Face API key
hf_token = os.getenv("HF_API_KEY")

# Modeli yükle (gated model ise token ver)
model = SentenceTransformer(
    "google/embeddinggemma-300m",
    token=hf_token
)

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]

# Embeddingleri al
embeddings = model.encode(sentences)

# Benzerlikleri cosine similarity ile hesapla
similarities = cosine_similarity(embeddings, embeddings)

print("Embeddings shape:", embeddings.shape)
print("Benzerlik matrisi:\n", similarities)


Embeddings shape: (4, 768)
Benzerlik matrisi:
 [[1.         0.91585195 0.98131907 0.7847327 ]
 [0.91585195 1.0000001  0.8955096  0.77843153]
 [0.98131907 0.8955096  0.99999976 0.7499244 ]
 [0.7847327  0.77843153 0.7499244  1.0000002 ]]


In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("google/embeddinggemma-300m")
vec = model.encode(["test"])
print(vec.shape)  # (1, 784) olmalı

(1, 768)


### chunk ederken cpu mu gpu mu kullanılıyor

In [8]:
import torch
print("CUDA kullanılabilir mi? ", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU adı:", torch.cuda.get_device_name(0))
else:
    print("CPU kullanılıyor")


CUDA kullanılabilir mi?  True
GPU adı: NVIDIA RTX A6000


In [7]:
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd
import tiktoken
import semchunk
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import numpy as np
import uuid
from typing import List, Dict
import os
from dataclasses import dataclass
import json
from dotenv import load_dotenv

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))

hf_token = os.getenv("HF_API_KEY")

# Konfigürasyon
@dataclass
class Config:
    # Google EmbeddingGemma ayarları
    MODEL_NAME: str = "google/embeddinggemma-300m"
    HF_TOKEN: str = hf_token

    # SemChunk ayarları
    TOKEN_SIZE: int = 512
    ENCODING_NAME: str = "cl100k_base"

    # Qdrant ayarları
    QDRANT_URL: str = "http://localhost:6333"
    COLLECTION_NAME: str = "gemma_semantic_chunks"
    DIMENSION: int = 768  # embeddinggemma-300m embedding boyutu

    # Dosya ayarları
    CSV_FILE: str = "/home/yapayzeka/ahsen_bulbul/data/yargitay_cleaned_2025-08-21.csv"
    BATCH_SIZE: int = 256

    # Cihaz ayarı
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"


class YargitaySemanticProcessor:
    def __init__(self, config: Config):
        self.config = config

        # SemChunk chunker
        self.encoding = tiktoken.get_encoding(config.ENCODING_NAME)
        self.chunker = semchunk.chunkerify(self.encoding, config.TOKEN_SIZE)

        # SentenceTransformer modeli (GPU/CPU seçimi burada yapılıyor)
        self.model = SentenceTransformer(
            config.MODEL_NAME,
            token=config.HF_TOKEN,
            device=config.DEVICE
        )
        print(f"✅ Model şu cihazda çalışıyor: {config.DEVICE}")

        # Qdrant client
        self.qdrant_client = QdrantClient(url=config.QDRANT_URL)

        print(f"✅ SemChunk hazır (Token boyutu: {config.TOKEN_SIZE})")
        print(f"✅ Google EmbeddingGemma modeli hazır")
        print(f"✅ Qdrant client hazır ({config.QDRANT_URL})")


True


### bir batch in embeddingleri oluştuktan sonra reduction yapıyo yani parça parça yapmış oluyor

In [9]:
def create_embeddings(self, texts: List[str], batch_size: int = 100) -> List[List[float]]:
        all_embeddings = []
        total = len(texts)
        for i in range(0, total, batch_size):
            batch_texts = texts[i:i + batch_size]

            start_embed = time.time()

            batch_embeddings = self.model.encode(batch_texts, show_progress_bar=True)

            end_embed = time.time()
            print(f"Batch embedding süresi: {end_embed - start_embed:.2f} saniye")

            pca=PCA(n_components=512)
            batch_embeddings_reduced = pca.fit_transform(batch_embeddings) #default 768 olan embedding boyutunu 512ye düşürüyoruz -> PCA
            all_embeddings.extend(batch_embeddings_reduced.tolist())

            print(f"  🔹 Embedding oluşturuldu: {i + len(batch_texts)}/{total}")
        return all_embeddings

### model training

In [12]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Modeli yükle
model = SentenceTransformer("google/embeddinggemma-300m")

# Küçük dataset
train_examples = [
    InputExample(texts=["araba", "otomobil"], label=1.0),
    InputExample(texts=["kedi", "köpek"], label=1.0),
    InputExample(texts=["masa", "sandalye"], label=1.0),
    InputExample(texts=["araba", "uçak"], label=0.0),
    InputExample(texts=["kedi", "araba"], label=0.0),
    InputExample(texts=["masa", "araba"], label=0.0),
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

# Loss fonksiyonu
train_loss = losses.CosineSimilarityLoss(model)

# Eğitim
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=10
)

# Test
emb1 = model.encode("araba")
emb2 = model.encode("otomobil")
emb3 = model.encode("kedi")

from sentence_transformers.util import cos_sim

print("Benzerlik (araba - otomobil):", cos_sim(emb1, emb2).item())
print("Benzerlik (araba - kedi):", cos_sim(emb1, emb3).item())


Step,Training Loss


Benzerlik (araba - otomobil): 0.547791063785553
Benzerlik (araba - kedi): -0.0018093292601406574
